Here we analyse the fixed points and their stability of the Lorenz - Saltzman model using a symbolic mathematical software  [Maxima](https://maxima.sourceforge.io/Maxima). It is based on the functional programming language [lisp](https://en.wikipedia.org/wiki/Lisp_(programming_language)). This is not the only software around. Wolfram Mathematica is a well-known, very powerful software under paid licence. Matlab also has a symbolic maths package; [SymPy](https://www.sympy.org/en/index.html) has reached a good level of maturity and presents the huge advantage to be in Python, giving access to the `Jupyter` notebook interface and the `matplotib`  interface. This is probably the recommended alternative for someone starting from scratch. Some more discussion of SymPy vs Maxima is available at https://github.com/sympy/sympy/wiki/SymPy-vs.-Maxima. 

We first define the system with the three time derivatives:


In [1]:
dX(X,Y,Z):=sigma * (Y-X);
dY(X,Y,Z):=X*(r - Z) - Y;
dZ(X,Y,Z):=X*Y - beta * Z;
assume(sigma>0, r>0, m>0);

(%o0)                    dX(X, Y, Z) := sigma (Y - X)

(%o1)                    dY(X, Y, Z) := X (r - Z) - Y

(%o2)                     dZ(X, Y, Z) := X Y - beta Z

(%o3)                      [sigma > 0, r > 0, m > 0]

Solving for all derivatives to be zero generates the fixedp points.

In [2]:
eq1: dX(X,Y,Z)=0;
eq2: dY(X,Y,Z)=0;
eq3: dZ(X,Y,Z)=0;

(%o4)                          (Y - X) sigma = 0

(%o5)                          X (r - Z) - Y = 0

(%o6)                          X Y - Z beta = 0

In [3]:
sols:solve([eq1,eq2,eq3],[X,Y,Z]);

(%o7) [[X = sqrt(beta r - beta), Y = sqrt(beta r - beta), Z = r - 1], 
[X = - sqrt(beta r - beta), Y = - sqrt(beta r - beta), Z = r - 1], 
[X = 0, Y = 0, Z = 0]]

We recognize the trivial fixed point, as well as the two symmetric fixed points on both sides of the $z-$axis, which  take real values only for $\rho>1$. 

The Jacobian of the derivatives is computed as follows:

In [4]:
J:jacobian([dX(X,Y,Z),dY(X,Y,Z),dZ(X,Y,Z)],[X,Y,Z]);

                          [ - sigma  sigma    0    ]
                          [                        ]
(%o8)                     [  r - Z    - 1    - X   ]
                          [                        ]
                          [    Y       X    - beta ]

We first evaluated the Jacobian at the trivial fixed point and consider its eigenvalues. `Maxima` gives us the three eigenvalues and their multiplicity. The `rhs` and `lhs` functions return the right-hand-side and left-hand-side of an expression, respectively

In [5]:
J0:psubst(sols[3],J);

                          [ - sigma  sigma    0    ]
                          [                        ]
(%o9)                     [    r      - 1     0    ]
                          [                        ]
                          [    0       0    - beta ]

In [6]:
simp:true$
eJ0:eigenvalues(J0);

                     2
           sqrt(sigma  + (4 r - 2) sigma + 1) + sigma + 1
(%o11) [[- ----------------------------------------------, 
                                 2
                      2
            sqrt(sigma  + (4 r - 2) sigma + 1) - sigma - 1
            ----------------------------------------------, - beta], [1, 1, 1]]
                                  2

We recognise the double root (first and second solutions), and the $-\beta$ eigenvalue along the $z$ axis. The trivial fixed point becomes a saddle when the first (and second) eigenvalue reach zero:

In [7]:
solve(eJ0[1][2],r);

(%o12)                              [r = 1]

Consider now the non-trivial fixed point, which become real for $r>1$.

In [8]:
assume(r>1);

(%o13)                              [r > 1]

In [9]:
J1:psubst(sols[1],J);

       [       - sigma               sigma                   0           ]
       [                                                                 ]
(%o14) [          1                   - 1          - sqrt(beta r - beta) ]
       [                                                                 ]
       [ sqrt(beta r - beta)  sqrt(beta r - beta)         - beta         ]

We could repeat the approach for the non-trivial fixed points and also ask `maxima` to compute the eigenvalues but this is unwise. The expressions are very heavy and we can do better. Heaving a computer algebra system should not stop us from thinking. Indeed, consider rather the characteristic polynomial $P(\lambda; \beta, \sigma, \rho)$, defined as the determinant $\text{det}(\lambda 1 - J)$ matrix. 


In [10]:
carpol:rat(determinant(diagmatrix(3,lambda)-J1), lambda);

             3                            2
(%o15) lambda  + (sigma + beta + 1) lambda  + (beta sigma + beta r) lambda
                                                    + (2 beta r - 2 beta) sigma

In [11]:
dcarpoldl:diff(carpol,lambda);

                2
(%o16)  3 lambda  + (2 sigma + 2 beta + 2) lambda + beta sigma + beta r

For positive values of the parameters, all coefficients of the polynomial are positive, and the theory of cubic equations tells us that, in this case, all roots _must_ be either real negative, or complex. It is easy enough to consider the case $r=1$, at which the roots are all real, and negative. By continuity, we expect all roots to be real negatives for $r$ slightly above 1. But they could become imaginary. 

This transition from negative real to imaginary roots occurs when two roots merge, and occur at an extrema of the $P(\lambda)$. Assuming $\beta$ and $\sigma$ given, the critical value of the parameter $r$ (called $r_1$) at which roots become complex must therefore satisfy $P(\lambda;r)=P^\prime(\lambda;r)=0$ (where the $^\prime$ symbol denotes a derivative with respect to $\lambda$). So we first solve $P^\prime(\lambda;r)$. 


In [12]:
SS:solve(dcarpoldl, lambda)$
SS[1];
SS[2];

                             2                                     2
(%o18) lambda = - (sqrt(sigma  + (2 - beta) sigma - 3 beta r + beta  + 2 beta
                                                     + 1) + sigma + beta + 1)/3

                           2                                     2
(%o19) lambda = (sqrt(sigma  + (2 - beta) sigma - 3 beta r + beta  + 2 beta
                                                     + 1) - sigma - beta - 1)/3

The roots naturally have the form $\lambda_\pm=a\pm \sqrt{b}$ and we inspect here the $\lambda_+$ solution (this is the solution for the local minimum, which comes as the second root; see graph below). We substitute this solution in the expression for $P(\lambda;r)$:

In [13]:
S3:ev(subst(SS[2], carpol), sigma=10, beta=8/3);

              961          41                             961          41 3
        (sqrt(--- - 8 r) - --) (8 r + 80)           (sqrt(--- - 8 r) - --)
               9           3                               9           3
(%o20) (--------------------------------- + 160 r + -----------------------
                        3                                      9
                                                     961          41 2
                                            41 (sqrt(--- - 8 r) - --)
                                                      9           3
                                          + -------------------------- - 160)/3
                                                        9

And the critical value for $r$ that we are looking for is the one that will make this expression equal to zero. As the algebraic expression is involved, we provide here a numerical estimate of the root. 

In [14]:
r1 : find_root(S3, r, 1, 3);

(%o21)                         1.345617179232956

We show the behaviour of $P(\lambda)$ for different values of $r$ and this will help us to visualise the specific behaviour at $r$:

In [20]:
r1;
setoption(legend, "r=0.9","r=1.3456","r=1.7")$
plot2d([ev(carpol, sigma=10, beta=8/3, r=0.9), 
        ev(carpol, sigma=10, beta=8/3, r=r1), 
        ev(carpol, sigma=10, beta=8/3, r=1.7)], 
        [lambda,-12,0], [legend,"r=0.9","r=r_crit","r=1.7"],
        [pdf_file, "/home/mcrucifix/Documents/Enseignement/LPHYS2114/Git/Jupyter/carpol.pdf"])$

(%o31)                         1.345617179232956


rat: replaced -5.333333333333332 by -16/3 = -5.333333333333333

rat: replaced 29.06666666666667 by 436/15 = 29.06666666666667

rat: replaced 18.43291622575767 by 86208740/4676891 = 18.43291622575767

rat: replaced 30.25497914462122 by 208574952/6893905 = 30.25497914462123

rat: replaced 37.33333333333334 by 112/3 = 37.33333333333334

rat: replaced 31.2 by 156/5 = 31.2


![](./carpol.pdf)

All we can say at this point is that for $r>r_1$ the roots  are complex, but numerical inspection shows that the points still act as attracting centers. They become repellers when the real part of the eigenvalue becomes positive. The critical value of the parameter $r$ from where instability emerges, called $r_H$, must then satisfy $P(i\omega;r)=0$, for real, non-zero $\omega$. This will be the boundary between negative and positive real parts.  The correspoding equation reads:


In [16]:
eqw:subst([lambda=sqrt(-1)*omega], carpol);

                                             2
(%o25) %i omega (beta sigma + beta r) - omega  (sigma + beta + 1)
                                                                              3
                                        + (2 beta r - 2 beta) sigma - %i omega

And given that all parameters are real, it actually generates two equations by considering together the real and imaginary parts.  Solving the combination gives:


In [17]:

SOL:solve([realpart(eqw), imagpart(eqw)], [r,omega])$
SOL[1];
SOL[2];

(%o27)                        [r = 1, omega = 0]

                 2
            sigma  + (beta + 3) sigma
(%o28) [r = -------------------------, 
                sigma - beta - 1
                                                                   beta
          omega = - sqrt(2) sqrt(sigma) sqrt(sigma + 1) sqrt(----------------)]
                                                             sigma - beta - 1

Again we have two roots but the trivial one $\omega=0$ does not interest us. The second is the relevant one, which we evaluate for the canonical parameers of the Lorenz model:

In [18]:
rh:ev(rhs(SOL[2][1]), sigma=10, beta=8/3);

                                      470
(%o29)                                ---
                                      19

In [19]:
float(rh);

(%o30)                         24.73684210526316

And allows us to conclude about the critical parameter $r_H$. 